In [1]:
import cv2
import numpy as np
import face_recognition
from PIL import Image
import glob
import re
import ntpath
import os
import dlib
import time
from imutils.face_utils import FaceAligner
import sqlite3
from imutils.face_utils import rect_to_bb
import imutils
from imutils import face_utils
import os
import shutil
import inception_resnet_v1
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
shutil.rmtree("../data/unknown_people/")

In [4]:
os.mkdir("../data/unknown_people/")

In [5]:
people = sqlite3.connect('../data/people.db')
c = people.cursor()

In [6]:
# show name people know name
for row in c.execute('select * from people_known order by old'):
    print(row)

OperationalError: no such table: people_known

In [6]:
# show name people in room equal empty
# c.execute('delete from people_inroom')
# people.commit()
for row in c.execute('select * from people_inroom order by old'):
    print(row)

('Dam Van Tai', 1, 24)


In [7]:
# show id people unknown name
# c.execute('delete from people_unknown')
# people.commit()
for row in c.execute('select * from people_unknown order by old'):
    print(row)

('unknown_0', 1, 33)


In [8]:
# recognition face center
def face_center(image):
#     detector = dlib.get_frontal_face_detector()
#     predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    rects = detector(image_gray, 1)
    for (i, rect) in enumerate(rects):
        points = predictor(image_gray, rect)
        points = face_utils.shape_to_np(points)
        center_face = points[37][1] - points[46][1]
#         cross_face1 = points[49][0] - points[58][0]
#         cross_face2 = points[58][0] - points[55][0]
#         print(cross_face1)
#         print(cross_face2)
#         cross = cross_face1 - cross_face2
#         print(cross)
#         if center_face < 10 & center_face > -10 & cross < 20 & cross > -20:
        if center_face < 10 & center_face > -10:
            return True
        else:
            return False

In [9]:
def _css_to_rect(css):
    """
    Convert a tuple in (top, right, bottom, left) order to a dlib `rect` object
    :param css:  plain tuple representation of the rect in (top, right, bottom, left) order
    :return: a dlib `rect` object
    """
    return dlib.rectangle(css[3], css[0], css[1], css[2])

In [10]:
tf.reset_default_graph() 
# sess = tf.InteractiveSession()
sess = tf.Session()
images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
images_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images_pl)
train_mode = tf.placeholder(tf.bool)
age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm, keep_probability=0.8,
                                                             phase_train=train_mode,
                                                             weight_decay=1e-5)

gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())
sess.run(init_op)
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state("../gender_age_tf/models/")
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("restore model!")
else:
    pass

INFO:tensorflow:Restoring parameters from ../gender_age_tf/models/model.ckpt-14001
restore model!


In [12]:
# def main(sess,age,gender,train_mode,images_pl):
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
name = ""
frame_number = 0
total_in_room = 0
total_known_in_room = 0
index = 0
faces = np.empty((1,160, 160, 3))

# Load known_face_encoding_array and known_face name from npy
known_face_encodings_array = np.load("../data/numpy/known_face_encoding.npy")
known_face_names = np.load("../data/numpy/known_face_names.npy")

# Convert nparray -> list 
number_person = len(known_face_encodings_array)
known_face_encodings_array = known_face_encodings_array.reshape(number_person, 128)
known_face_encodings = []
for i in range(len(known_face_encodings_array)):
    known_face_encodings.append(known_face_encodings_array[i])

# face encodings of people in room
face_encodings_in_room = []
face_names_in_room = []
face_encodings_unknown_in_room = []
face_names_unknown_in_room = []
# known_face_encodings: array known_face_encodings 

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=160)

# Capture from camera of own computer
# video_capture = cv2.VideoCapture(0)

# Capture from camera of extern computer
video_capture_extern = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
#     ret, frame = video_capture.read()    
    ret_extern, frame_extern = video_capture_extern.read()
    
    frame_number += 1

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
#     rgb_frame = frame[:, :, ::-1]
    rgb_frame_extern = frame_extern[:, :, ::-1]
    
    # Use computer configuration low
#     if (frame_number % 3 == 0):

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame_extern)
#         print(face_locations)
    face_encodings = face_recognition.face_encodings(rgb_frame_extern, face_locations)

    face_names = []
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        face_unknown = frame_extern[top:bottom, left:right]
#         face_unknown = cv2.cvtColor(face_unknown, cv2.COLOR_BGR2RGB)
#         face_unknown_gray = cv2.cvtColor(face_unknown, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.cvtColor(rgb_frame_extern, cv2.COLOR_BGR2GRAY)
        # test face center to process
        if face_center(face_unknown) == True:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(face_encodings_in_room, face_encoding, tolerance=0.4)

#             print(face_recognition.face_distance(known_face_encodings, face_encoding))
#             print(matches)
#             print(np.min(face_recognition.face_distance(known_face_encodings, face_encoding)))

            distance = face_recognition.face_distance(known_face_encodings, face_encoding)
            point = np.min(distance)
            print("point known face encoding in face center ", point)
            index_point_min = np.argmin(distance)
            name_face = face_names_in_room[index_point_min]
            
            c.execute("delete from people_inroom where name=?", (name_face,))
            people.commit()
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
people.close()

point known face encoding in face center  0.394357258312


IndexError: list index out of range